## Masalah 2 : Minimum Spanning Tree 38 Provinsi di Indonesia

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import heapq
import folium
from folium.plugins import TimestampedGeoJson
import os
import time
from datetime import datetime, timedelta
from IPython.display import display

def get_province_coords_from_shapefile(shapefile_path):
    """
    Membaca shapefile, memfilter untuk Indonesia, menghitung titik tengah
    (centroid) setiap provinsi, dan menambahkan provinsi baru secara manual.
    """
    try:
        world_map = gpd.read_file(shapefile_path)
        indonesia_map = world_map[world_map['iso_a2'] == 'ID'].copy()
        province_name_col = 'NAME_1' if 'NAME_1' in indonesia_map.columns else 'name'
        indonesia_map['centroid'] = indonesia_map['geometry'].centroid
        coords = {row[province_name_col]: (row['centroid'].y, row['centroid'].x) for _, row in indonesia_map.iterrows()}
        print(f"Berhasil mendapatkan {len(coords)} koordinat provinsi dari shapefile.")
    except Exception as e:
        print(f"Gagal membaca shapefile: {e}. Menggunakan dictionary kosong.")
        coords = {}

    # Menambahkan 5 provinsi baru secara manual karena koordinat provinsi-provinsi 
    # ini belum tersedia pada dataset
    new_provinces = {
        'Kalimantan Utara': (2.916, 117.361),     # Tanjung Selor
        'Papua Selatan': (-8.499, 140.402),       # Merauke
        'Papua Tengah': (-3.367, 135.493),        # Nabire
        'Papua Pegunungan': (-4.148, 138.948),    # Wamena (Jayawijaya)
        'Papua Barat Daya': (-0.882, 131.258)     # Sorong
    }

    # Menggabungkan data dari shapefile dengan data manual
    coords.update(new_provinces)

    # Menyesuaikan nama provinsi yang mungkin tidak konsisten
    # Contoh: "Papua" di shapefile mungkin merujuk pada provinsi induk sebelum pemekaran.
    # Kita asumsikan nama 'Papua' dari shapefile adalah untuk Provinsi Papua (Jayapura)
    # Jika nama 'Irian Jaya Barat' ada, ganti menjadi 'Papua Barat'
    if 'Irian Jaya Barat' in coords:
        coords['Papua Barat'] = coords.pop('Irian Jaya Barat')


    print(f"Total provinsi setelah penambahan manual: {len(coords)}.")
    return coords


def haversine(lat1, lon1, lat2, lon2): # rumus jarak permukaan bumi
    R = 6371; lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2]); dlon = lon2 - lon1; dlat = lat2 - lat1; a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2; c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)); return R * c

def build_graph(coords):
    provinces = list(coords.keys()); n = len(provinces); province_to_idx = {p: i for i, p in enumerate(provinces)}; adj_matrix = np.zeros((n, n));
    for i in range(n):
        for j in range(i, n): p1, p2 = provinces[i], provinces[j]; dist = haversine(coords[p1][0], coords[p1][1], coords[p2][0], coords[p2][1]); adj_matrix[i, j] = adj_matrix[j, i] = dist
    return provinces, province_to_idx, adj_matrix

class DSU: # Disjoint Set Union Data Structure
    def __init__(self, n): self.parent = list(range(n))
    def find(self, i):
        if self.parent[i] == i: return i
        self.parent[i] = self.find(self.parent[i]); return self.parent[i]
    def union(self, i, j):
        root_i, root_j = self.find(i), self.find(j);
        if root_i != root_j: self.parent[root_i] = root_j; return True
        return False


In [2]:
def generate_mst_animation_steps(adj_matrix, algorithm='kruskal'):
    n = len(adj_matrix)
    animation_steps = []
    total_distance = 0
    time_step = 0
    base_time = datetime(2025, 1, 1)

    if algorithm == 'prim':
        visited = [False] * n
        min_heap = [(0, -1, 0)]
        while min_heap and len(animation_steps) < n - 1:
            weight, u, v = heapq.heappop(min_heap)
            if visited[v]: continue
            visited[v] = True
            if u != -1:
                total_distance += weight
                current_time = base_time + timedelta(hours=time_step)
                animation_steps.append({'edge': (u, v), 'time': current_time.isoformat() + "Z"})
                time_step += 1
            for neighbor in range(n):
                if not visited[neighbor]: heapq.heappush(min_heap, (adj_matrix[v][neighbor], v, neighbor))
    else: # Kruskal
        edges = sorted([(adj_matrix[i][j], i, j) for i in range(n) for j in range(i + 1, n)])
        dsu = DSU(n)
        for weight, u, v in edges:
            if dsu.union(u, v):
                total_distance += weight
                current_time = base_time + timedelta(hours=time_step)
                animation_steps.append({'edge': (u, v), 'time': current_time.isoformat() + "Z"})
                time_step += 1
                if len(animation_steps) == n - 1: break
    return animation_steps, total_distance
def plot_animated_mst(animation_steps, provinces, coords, title):
    map_center = [-2.5489, 118.0149]
    f_map = folium.Map(location=map_center, zoom_start=5)

    for province_name, (lat, lon) in coords.items():
        folium.Marker(
            location=[lat, lon],
            popup=f"<strong>{province_name}</strong>",
            tooltip=province_name,
            icon=folium.Icon(color='darkblue', icon='info-sign')
        ).add_to(f_map)

    features = []
    for step in animation_steps:
        u, v = step['edge']
        p1_name, p2_name = provinces[u], provinces[v]

        if p1_name in coords and p2_name in coords:
            loc1, loc2 = coords[p1_name], coords[p2_name]
            features.append({
                'type': 'Feature',
                'geometry': {
                    'type': 'LineString',
                    'coordinates': [[loc1[1], loc1[0]], [loc2[1], loc2[0]]],
                },
                'properties': {
                    'times': [step['time'], step['time']],
                    'style': {'color': '#FF5733', 'weight': 3.5},
                    'popup': f'{p1_name} - {p2_name}'
                }
            })

    if features:
        TimestampedGeoJson(
            {'type': 'FeatureCollection', 'features': features},
            period='PT1H',
            add_last_point=True,
            auto_play=False,
            loop=False,
            max_speed=2,
            loop_button=True,
            time_slider_drag_update=True
        ).add_to(f_map)

    title_html = f'<h3 align="center" style="font-size:16px"><b>{title}</b></h3>'
    f_map.get_root().html.add_child(folium.Element(title_html))
    return f_map


## Animasi Algoritma MST

In [3]:
def generate_mst_animation_steps(adj_matrix, algorithm='kruskal'):
    n = len(adj_matrix)
    animation_steps = []
    total_distance = 0
    time_step = 0
    base_time = datetime(2025, 1, 1)

    if algorithm == 'prim':
        visited = [False] * n
        min_heap = [(0, -1, 0)]
        while min_heap and len(animation_steps) < n - 1:
            weight, u, v = heapq.heappop(min_heap)
            if visited[v]: continue
            visited[v] = True
            if u != -1:
                total_distance += weight
                current_time = base_time + timedelta(hours=time_step)
                animation_steps.append({'edge': (u, v), 'time': current_time.isoformat() + "Z"})
                time_step += 1
            for neighbor in range(n):
                if not visited[neighbor]: heapq.heappush(min_heap, (adj_matrix[v][neighbor], v, neighbor))
    else: # Kruskal
        edges = sorted([(adj_matrix[i][j], i, j) for i in range(n) for j in range(i + 1, n)])
        dsu = DSU(n)
        for weight, u, v in edges:
            if dsu.union(u, v):
                total_distance += weight
                current_time = base_time + timedelta(hours=time_step)
                animation_steps.append({'edge': (u, v), 'time': current_time.isoformat() + "Z"})
                time_step += 1
                if len(animation_steps) == n - 1: break
    return animation_steps, total_distance
def plot_animated_mst(animation_steps, provinces, coords, title):
    map_center = [-2.5489, 118.0149]
    f_map = folium.Map(location=map_center, zoom_start=5)

    for province_name, (lat, lon) in coords.items():
        folium.Marker(
            location=[lat, lon],
            popup=f"<strong>{province_name}</strong>",
            tooltip=province_name,
            icon=folium.Icon(color='darkblue', icon='info-sign')
        ).add_to(f_map)

    features = []
    for step in animation_steps:
        u, v = step['edge']
        p1_name, p2_name = provinces[u], provinces[v]

        if p1_name in coords and p2_name in coords:
            loc1, loc2 = coords[p1_name], coords[p2_name]
            features.append({
                'type': 'Feature',
                'geometry': {
                    'type': 'LineString',
                    'coordinates': [[loc1[1], loc1[0]], [loc2[1], loc2[0]]],
                },
                'properties': {
                    'times': [step['time'], step['time']],
                    'style': {'color': '#FF5733', 'weight': 3.5},
                    'popup': f'{p1_name} - {p2_name}'
                }
            })

    if features:
        TimestampedGeoJson(
            {'type': 'FeatureCollection', 'features': features},
            period='PT1H',
            add_last_point=True,
            auto_play=False,
            loop=False,
            max_speed=2,
            loop_button=True,
            time_slider_drag_update=True
        ).add_to(f_map)

    title_html = f'<h3 align="center" style="font-size:16px"><b>{title}</b></h3>'
    f_map.get_root().html.add_child(folium.Element(title_html))
    return f_map


In [4]:
SHAPEFILE_PATH = '../data/ne_10m_admin_1_states_provinces.shp'

provinsi_coords = get_province_coords_from_shapefile(SHAPEFILE_PATH)

if provinsi_coords:
  provinces, province_to_idx, adj_matrix = build_graph(provinsi_coords)

# --- Algoritma Prim ---
start_time_prim = time.time()
prim_anim_steps, prim_total_distance = generate_mst_animation_steps(adj_matrix, algorithm='prim')
end_time_prim = time.time()
execution_time_prim = end_time_prim - start_time_prim

prim_anim_map = plot_animated_mst(prim_anim_steps, provinces, provinsi_coords, "Proses Pembentukan MST (38 Provinsi) - Algoritma Prim")

print(f"Total Jarak MST (Prim): {prim_total_distance:.2f} km")
print(f"Waktu eksekusi (Prim): {execution_time_prim:.6f} detik")

prim_anim_map.save("prim_mst_animation_38.html")
print("Animasi Prim disimpan ke prim_mst_animation_38.html")
display(prim_anim_map)
print("\n" + "="*60 + "\n")

# --- Algoritma Kruskal ---
start_time_kruskal = time.time()
kruskal_anim_steps, kruskal_total_distance = generate_mst_animation_steps(adj_matrix, algorithm='kruskal')
end_time_kruskal = time.time()
execution_time_kruskal = end_time_kruskal - start_time_kruskal

kruskal_anim_map = plot_animated_mst(kruskal_anim_steps, provinces, provinsi_coords, "Proses Pembentukan MST (38 Provinsi) - Algoritma Kruskal")

print(f"Total Jarak MST (Kruskal): {kruskal_total_distance:.2f} km")
print(f"Waktu eksekusi (Kruskal): {execution_time_kruskal:.6f} detik")

kruskal_anim_map.save("kruskal_mst_animation_38.html")
print("Animasi Kruskal disimpan ke kruskal_mst_animation_38.html")
display(kruskal_anim_map)


C:\Users\rendy\AppData\Local\Temp\ipykernel_221632\3835925726.py:21: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  indonesia_map['centroid'] = indonesia_map['geometry'].centroid


Berhasil mendapatkan 33 koordinat provinsi dari shapefile.
Total provinsi setelah penambahan manual: 38.
Total Jarak MST (Prim): 10650.15 km
Waktu eksekusi (Prim): 0.001000 detik
Animasi Prim disimpan ke prim_mst_animation_38.html




Total Jarak MST (Kruskal): 10650.15 km
Waktu eksekusi (Kruskal): 0.001699 detik
Animasi Kruskal disimpan ke kruskal_mst_animation_38.html
